# 01 — PDB I/O, Visualization, and Structural Comparison (Colab)

**Goals**
- Download and read PDB files
- Inspect basic metadata (chains, residues, ligands)
- Visualize structures in Colab
- Align two structures and compute RMSD

**NEW in this version**
- Sequence-align two related proteins of different length (**4EY7** vs **6XYS**, acetylcholinesterase)
- Use the alignment to identify overlapping Cα atoms in Chain A
- Superpose + compute RMSD on the overlapping set
- Visualize the best superposition in **py3Dmol**


## 0) Setup (install packages)

In [3]:
# If you're using a course repo later, you can replace this with a git-clone + requirements install.
!pip -q install biopython mdtraj py3Dmol requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 99.3 MB/s eta 0:00:00


In [4]:
import os, sys
import numpy as np
import requests
from pathlib import Path

print("Python:", sys.version.split()[0])
ROOT = Path("/content/structbio")
DATA = ROOT / "data"
OUT  = ROOT / "outputs"
for d in [DATA, OUT]:
    d.mkdir(parents=True, exist_ok=True)
print("DATA:", DATA)
print("OUT :", OUT)

Python: 3.12.12
DATA: /content/structbio/data
OUT : /content/structbio/outputs


## 1) Download a PDB file (and cache it)

In [5]:
def fetch_pdb(pdb_id: str, out_dir: Path = DATA) -> Path:
    """Download a PDB from RCSB and save it locally. Returns the local path."""
    pdb_id = pdb_id.upper()
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"{pdb_id}.pdb"
    if out_path.exists() and out_path.stat().st_size > 0:
        print(f"Using cached: {out_path}")
        return out_path

    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    out_path.write_text(r.text)
    print(f"Downloaded: {out_path}")
    return out_path

pdb1_path = fetch_pdb("1CRN")  # small protein: crambin
pdb2_path = fetch_pdb("1EJG")  # another small protein for comparison
pdb1_path, pdb2_path

Downloaded: /content/structbio/data/1CRN.pdb
Downloaded: /content/structbio/data/1EJG.pdb


(PosixPath('/content/structbio/data/1CRN.pdb'),
 PosixPath('/content/structbio/data/1EJG.pdb'))

## 2) Read and inspect PDB content with Biopython

In [7]:
from Bio.PDB import PDBParser

parser = PDBParser(QUIET=True)
structure1 = parser.get_structure("pdb1", str(pdb1_path))
structure2 = parser.get_structure("pdb2", str(pdb2_path))

def summarize_structure(structure):
    models = list(structure.get_models())
    chains = list(structure.get_chains())
    residues = [r for r in structure.get_residues()]
    atoms = list(structure.get_atoms())

    # Basic counts
    print(f"Models:   {len(models)}")
    print(f"Chains:   {len(chains)} -> {[c.id for c in chains]}")
    print(f"Residues: {len(residues)}")
    print(f"Atoms:    {len(atoms)}")

    # Identify hetero residues (ligands, ions, waters)
    hetero = []
    waters = 0
    for r in residues:
        hetflag, resseq, icode = r.get_id()
        if str(hetflag).startswith("W"):
            waters += 1
        elif str(hetflag).strip() != "":
            hetero.append(r)
    if hetero:
        names = sorted({r.get_resname() for r in hetero})
        print(f"Hetero residues (non-water): {len(hetero)} -> {names}")
    print(f"Waters: {waters}")

print("=== PDB1:", pdb1_path.name, "===")
summarize_structure(structure1)

print("\n=== PDB2:", pdb2_path.name, "===")
summarize_structure(structure2)

=== PDB1: 1CRN.pdb ===
Models:   1
Chains:   1 -> ['A']
Residues: 46
Atoms:    327
Waters: 0

=== PDB2: 1EJG.pdb ===
Models:   1
Chains:   1 -> ['A']
Residues: 46
Atoms:    641
Waters: 0


### Extract a chain sequence (roughly)

In [8]:
from Bio.PDB.Polypeptide import PPBuilder

ppb = PPBuilder()

def get_chain_sequences(structure):
    seqs = {}
    for model in structure:
        for chain in model:
            peptides = ppb.build_peptides(chain)
            if not peptides:
                continue
            # Many PDBs have multiple peptide segments; concatenate for simplicity
            seq = "".join(str(p.get_sequence()) for p in peptides)
            seqs[chain.id] = seq
        break  # first model
    return seqs

seqs1 = get_chain_sequences(structure1)
seqs2 = get_chain_sequences(structure2)

print("PDB1 sequences:")
for ch, seq in seqs1.items():
    print(ch, seq[:80] + ("..." if len(seq) > 80 else ""), f"(len={len(seq)})")
print("\nPDB2 sequences:")
for ch, seq in seqs2.items():
    print(ch, seq[:80] + ("..." if len(seq) > 80 else ""), f"(len={len(seq)})")

PDB1 sequences:
A TTCCPSIVARSNFNVCRLPGTPEAICATYTGCIIIPGATCPGDYAN (len=46)

PDB2 sequences:
A TTCCPSIVARSNFNVCRLPGTPEAICATYTGCIIIPGATCPGDYAN (len=46)


## 3) Visualize in Colab using `py3Dmol`

In [9]:
import py3Dmol

def show_pdb(pdb_path: Path, style="cartoon", color="spectrum", width=650, height=450):
    pdb_txt = pdb_path.read_text()
    view = py3Dmol.view(width=width, height=height)
    view.addModel(pdb_txt, "pdb")
    if style == "cartoon":
        view.setStyle({"cartoon": {"color": color}})
    elif style == "stick":
        view.setStyle({"stick": {}})
    else:
        view.setStyle({style: {}})
    view.zoomTo()
    return view

show_pdb(pdb1_path, style="cartoon").show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [10]:
show_pdb(pdb2_path, style="cartoon").show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Optional: highlight ligands/hetero atoms

---



In [15]:
def show_with_hetero(pdb_path: Path, width=650, height=450):
    pdb_txt = pdb_path.read_text()
    view = py3Dmol.view(width=width, height=height)
    view.addModel(pdb_txt, "pdb")
    # Protein cartoon
    view.setStyle({"protein": {}}, {"cartoon": {"color": "spectrum"}})
    # Hetero as sticks (includes ions/ligands)
    view.setStyle({"hetflag": True}, {"stick": {"radius": "10"}})
    view.zoomTo()
    return view

show_with_hetero(pdb1_path).show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## 4) RMSD alignment when structures match (MDTraj quick path)

This section works best when the structures have **the same residues** (same protein, same construct).
For related proteins with **insertions/deletions**, skip to **Section 5** (sequence-alignment-guided RMSD).


In [16]:
import mdtraj as md

t1 = md.load(str(pdb1_path))
t2 = md.load(str(pdb2_path))

sel1 = t1.topology.select("name CA and protein")
sel2 = t2.topology.select("name CA and protein")
print("CA counts:", len(sel1), len(sel2))

CA counts: 46 46


In [17]:
def ca_rmsd_after_alignment(t_ref, t_mobile):
    sel_ref = t_ref.topology.select("name CA and protein")
    sel_mob = t_mobile.topology.select("name CA and protein")
    if len(sel_ref) != len(sel_mob):
        raise ValueError(
            f"CA atom counts differ ({len(sel_ref)} vs {len(sel_mob)}). "
            "Use Section 5 for sequence-alignment-guided RMSD."
        )
    ref = t_ref[0] if t_ref.n_frames > 1 else t_ref
    mob = t_mobile[0] if t_mobile.n_frames > 1 else t_mobile
    mob = mob.slice(np.arange(mob.n_frames))  # copy-like
    mob.superpose(ref, atom_indices=sel_mob, ref_atom_indices=sel_ref)
    rmsd_nm = md.rmsd(mob, ref, atom_indices=sel_mob, ref_atom_indices=sel_ref)
    return float(rmsd_nm[0] * 10.0)  # Å

try:
    print("Cα RMSD:", ca_rmsd_after_alignment(t1, t2), "Å")
except Exception as e:
    print("(As expected) RMSD not computed here:", e)

Cα RMSD: 0.10557249188423157 Å


## 5) Sequence-alignment-guided RMSD for proteins of different length

We now compare two acetylcholinesterase structures from different species:

- **4EY7** (Chain A)
- **6XYS** (Chain A)

These have **different numbers of residues**, so we:
1. Extract Chain A sequences **from the PDB coordinates** (only residues present)
2. Do a **global sequence alignment** (Biopython)
3. Use the alignment to identify **matched residue pairs**
4. Collect overlapping **Cα atoms** for those matched residues
5. Compute best-fit superposition + RMSD (Biopython `Superimposer`)
6. Visualize the superposition in **py3Dmol**


In [18]:
# Download the two AChE structures
pdb_ref_path = fetch_pdb("4EY7")
pdb_mob_path = fetch_pdb("6XYS")  # we'll align this onto 4EY7

from Bio.PDB import PDBParser
parser = PDBParser(QUIET=True)
ref_struct = parser.get_structure("4EY7", str(pdb_ref_path))
mob_struct = parser.get_structure("6XYS", str(pdb_mob_path))

# Grab first model, chain A
ref_chain = next(ref_struct.get_models()).child_dict["A"]
mob_chain = next(mob_struct.get_models()).child_dict["A"]

print("4EY7 Chain A residues (incl het/water):", len(list(ref_chain.get_residues())))
print("6XYS Chain A residues (incl het/water):", len(list(mob_chain.get_residues())))

Downloaded: /content/structbio/data/4EY7.pdb
Downloaded: /content/structbio/data/6XYS.pdb
4EY7 Chain A residues (incl het/water): 822
6XYS Chain A residues (incl het/water): 631


### 5.1 Extract the *coordinate-present* protein sequence with residue mapping

In [22]:
import Bio.PDB.Polypeptide
help(Bio.PDB.Polypeptide)

Help on module Bio.PDB.Polypeptide in Bio.PDB:

NAME
    Bio.PDB.Polypeptide - Polypeptide-related classes (construction and representation).

DESCRIPTION
    Simple example with multiple chains,

        >>> from Bio.PDB.PDBParser import PDBParser
        >>> from Bio.PDB.Polypeptide import PPBuilder
        >>> structure = PDBParser().get_structure('2BEG', 'PDB/2BEG.pdb')
        >>> ppb=PPBuilder()
        >>> for pp in ppb.build_peptides(structure):
        ...     print(pp.get_sequence())
        LVFFAEDVGSNKGAIIGLMVGGVVIA
        LVFFAEDVGSNKGAIIGLMVGGVVIA
        LVFFAEDVGSNKGAIIGLMVGGVVIA
        LVFFAEDVGSNKGAIIGLMVGGVVIA
        LVFFAEDVGSNKGAIIGLMVGGVVIA

    Example with non-standard amino acids using HETATM lines in the PDB file,
    in this case selenomethionine (MSE):

        >>> from Bio.PDB.PDBParser import PDBParser
        >>> from Bio.PDB.Polypeptide import PPBuilder
        >>> structure = PDBParser().get_structure('1A8O', 'PDB/1A8O.pdb')
        >>> ppb=PPBuilder

In [24]:
from Bio.PDB.Polypeptide import is_aa, three_to_index, index_to_one

def chain_sequence_and_residues(chain):
    """Return (sequence_string, residue_list) for protein residues with coordinates."""
    seq = []
    residues = []
    for res in chain.get_residues():
        if not is_aa(res, standard=True):
            continue
        # Require CA to exist for our downstream mapping
        if "CA" not in res:
            continue
        try:
            aa = index_to_one( three_to_index(res.get_resname()) )
        except KeyError:
            aa = "X"
        seq.append(aa)
        residues.append(res)
    return "".join(seq), residues

ref_seq, ref_res_list = chain_sequence_and_residues(ref_chain)
mob_seq, mob_res_list = chain_sequence_and_residues(mob_chain)

print("4EY7(A) sequence length (present in coords):", len(ref_seq))
print("6XYS(A) sequence length (present in coords):", len(mob_seq))
print("4EY7(A) first 80:", ref_seq[:80])
print("6XYS(A) first 80:", mob_seq[:80])

print('\nNOTE these are NOT aligned!')

4EY7(A) sequence length (present in coords): 530
6XYS(A) sequence length (present in coords): 535
4EY7(A) first 80: EDAELLVTVRGGRLRGIRLKTPGGPVSAFLGIPFAEPPMGPRRFLPPEPKQPWSGVVDATTFQSVCYQYVDTLYPGFEGT
6XYS(A) first 80: DRLVVQTSSGPVRGRSVTVQGREVHVYTGIPYAKPPVEDLRFRKPVPAEPWHGVLDATGLSATCVQERYEYFPGFSGEEI

NOTE these are NOT aligned!


### 5.2 Global sequence alignment (Biopython)

In [25]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

# Global alignment with reasonable scoring (tweak if you like)
aln = pairwise2.align.globalms(ref_seq, mob_seq, 2, -1, -10, -0.5, one_alignment_only=True)[0]
aln_ref, aln_mob, score, begin, end = aln

print("Alignment score:", score)
print(format_alignment(*aln))

Alignment score: 6.5
EDAELLVTVRGGRLRGIRLKTPGGPVSAFLGIPFAEPPMGPRRFLPPEPKQPWSGVVDATTFQSVCYQYVDTLYPGFEGTEMWNPNRELSEDCLYLNVWTPYPRPTSPTPVLVWIYGGGFYSGASSLDVYDGRFLVQAERTVLVSMNYRVGAFGFLAL------PGSREAPGNVGLLDQRLALQWVQENVAAFGGDPTSVTLFGESAGAASVGMHLLSPPSRGLFHRAVLQSGAPNGPWATVGMGEARRRATQLAHLVGCPNDTELVACLRTRPAQVLVNHEWHVLPQESV---------FRFSFVPVVDGDFLSDTPEALINAGDFHGLQVLVGVVKDEGSYFLVYGAPG-FSKDNESLISRAEFLAGVRVGVPQVSDLAAEAVVLHYTDWLHPEDPARLREALSDVVGDHNVVCPVAQLAGRLAAQGARVYAYVFEHRASTLSWPLWMGVPHGYEIEFIFGIPLDPSRNYTAEEKIFAQRLMRYWANFARTGDPNEPRDPQWPPYTAGAQQYV-SLDLRPLEVRRGLRAQACAFWNRFLPKLLSA
  ..|.|....|..||......|..|....|||.|.||....||..|.|..||.||.|||.....|.|......|||.|.|.||||...||||||.|||    .|....|.|.|||||||..|...||.|..............|..|||||||||.|      ....||||||||.||.||..|...|..||||.|...||||||||..||...|.||..|||..|...|||..|.||.......|......|.....|     ....|.|.||.|............||         ..|...|..||.||...|..|....|......|.|.|.|||.|||.|.... |.||......|...|...............||....||.| ...............||||...||....|..||..||.

/usr/local/lib/python3.12/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


### 5.3 Convert alignment into matched residue pairs

In [45]:
def alignment_to_residue_pairs(aln_ref: str, aln_mob: str, ref_res: list, mob_res: list):
    """Map aligned letters to residue objects; return list of (ref_res, mob_res) matched positions."""
    pairs = []
    i_ref = 0
    i_mob = 0
    for a, b in zip(aln_ref, aln_mob):
        if a != "-" and b != "-":
            pairs.append((ref_res[i_ref], mob_res[i_mob]))
        if a != "-":
            i_ref += 1
        if b != "-":
            i_mob += 1
    return pairs

pairs = alignment_to_residue_pairs(aln_ref, aln_mob, ref_res_list, mob_res_list)
print("Matched residue pairs:", len(pairs))

def res_id(res):
    hetflag, resseq, icode = res.get_id()
    return f"{res.get_resname()} {resseq}{icode.strip() or ''}"

# show matches for first n_match
n_match = 50
for k in range(n_match):
    print(res_id(pairs[k][0]), "<->", res_id(pairs[k][1]))

Matched residue pairs: 518
ALA 6 <-> ASP 3
GLU 7 <-> ARG 4
LEU 8 <-> LEU 5
LEU 9 <-> VAL 6
VAL 10 <-> VAL 7
THR 11 <-> GLN 8
VAL 12 <-> THR 9
ARG 13 <-> SER 10
GLY 14 <-> SER 11
GLY 15 <-> GLY 12
ARG 16 <-> PRO 13
LEU 17 <-> VAL 14
ARG 18 <-> ARG 15
GLY 19 <-> GLY 16
ILE 20 <-> ARG 17
ARG 21 <-> SER 18
LEU 22 <-> VAL 19
LYS 23 <-> THR 20
THR 24 <-> VAL 21
PRO 25 <-> GLN 22
GLY 26 <-> GLY 23
GLY 27 <-> ARG 24
PRO 28 <-> GLU 25
VAL 29 <-> VAL 26
SER 30 <-> HIS 27
ALA 31 <-> VAL 28
PHE 32 <-> TYR 29
LEU 33 <-> THR 30
GLY 34 <-> GLY 31
ILE 35 <-> ILE 32
PRO 36 <-> PRO 33
PHE 37 <-> TYR 34
ALA 38 <-> ALA 35
GLU 39 <-> LYS 36
PRO 40 <-> PRO 37
PRO 41 <-> PRO 38
MET 42 <-> VAL 39
GLY 43 <-> GLU 40
PRO 44 <-> ASP 41
ARG 45 <-> LEU 42
ARG 46 <-> ARG 43
PHE 47 <-> PHE 44
LEU 48 <-> ARG 45
PRO 49 <-> LYS 46
PRO 50 <-> PRO 47
GLU 51 <-> VAL 48
PRO 52 <-> PRO 49
LYS 53 <-> ALA 50
GLN 54 <-> GLU 51
PRO 55 <-> PRO 52


### 5.4 Build overlapping Cα atom lists and superpose

In [59]:
from Bio.PDB import Superimposer

ref_atoms = []
mob_atoms = []
for r_ref, r_mob in pairs:
    if "CA" in r_ref and "CA" in r_mob:
        ref_atoms.append(r_ref["CA"])
        mob_atoms.append(r_mob["CA"])

print("Overlapping CA atoms used ref :", len(ref_atoms))
print("Overlapping CA atoms used mob", len(mob_atoms))

sup = Superimposer()
sup.set_atoms(ref_atoms, mob_atoms)  # sets rotation/translation to put mob onto ref
print(f"RMSD over matched Cα set: {sup.rms:.3f} Å")

Overlapping CA atoms used ref : 518
Overlapping CA atoms used mob 518
RMSD over matched Cα set: 4.257 Å


### 5.5 Apply the transform to the entire mobile structure and save new PDBs

In [35]:
rot, tran = sup.rotran
for atom in mob_struct.get_atoms():
    atom.transform(rot, tran)

aligned_mob_path = OUT / "6XYS_chainA_aligned_to_4EY7.pdb"
ref_out_path      = OUT / "4EY7_chainA_ref.pdb"

from Bio.PDB import PDBIO, Select

class ChainSelect(Select):
    def __init__(self, chain_id: str):
        self.chain_id = chain_id
    def accept_chain(self, chain):
        return 1 if chain.id == self.chain_id else 0

io = PDBIO()
io.set_structure(ref_struct)
io.save(str(ref_out_path), select=ChainSelect("A"))

io.set_structure(mob_struct)
io.save(str(aligned_mob_path), select=ChainSelect("A"))

print("Wrote:", ref_out_path)
print("Wrote:", aligned_mob_path)

Wrote: /content/structbio/outputs/4EY7_chainA_ref.pdb
Wrote: /content/structbio/outputs/6XYS_chainA_aligned_to_4EY7.pdb


### 5.6 Visualize the best superposition (two models) in py3Dmol

In [58]:
import py3Dmol
from pathlib import Path

ref_txt = Path(ref_out_path).read_text()
mob_txt = Path(aligned_mob_path).read_text()

view = py3Dmol.view(width=750, height=520)
view.addModel(ref_txt, "pdb")   # model 0 (reference)
view.addModel(mob_txt, "pdb")   # model 1 (aligned mobile)

view.setStyle({"model": 0}, {"cartoon": {"color": "blue"}})
view.setStyle({"model": 1}, {"cartoon": {"color": "red"}})

# Optional: show matched CA atoms as spheres (first ~200 to keep it light)
max_spheres = 500
for a in ref_atoms[:max_spheres]:
    x, y, z = a.get_coord()
    # print('xyz', x,y,z)
    view.addSphere({"center": {"x": float(x), "y": float(y), "z": float(z)},
                    "radius": 1.0, "color": "yellow", "opacity": 0.8})

view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

From: https://biopython.org/docs/dev/Tutorial/chapter_pdb.html#aligning-dissimilar-structures

###  Aligning dissimilar structures

f you want to align two structures with low sequence identity (less than 50%) you can use the CEAligner class, which implements the Combinatorial Extension (CE) algorithm for structural alignment. This method automatically finds the best matching regions between two structures and superimposes them by using the C-alpha coordinates (for proteins) or C4’ (for nucleic acids)

The algorithm used in the CEAligner class is from Shindyalov & Bourne [[Shindyalov1998]](https://biopython.org/docs/dev/Tutorial/chapter_bibliography.html#shindyalov1998) and uses the QCPSuperimposer to perform the actual superimposition after discovering the best matching regions



In [88]:
from Bio.PDB import PDBParser
parser = PDBParser(QUIET=True)
ref_struct = parser.get_structure("4EY7", str(pdb_ref_path))
mob_struct = parser.get_structure("6XYS", str(pdb_mob_path))

from Bio.PDB.cealign import CEAligner
aligner = CEAligner()
aligner.set_reference(ref_struct)
aligner.align(mob_struct)

# Get RMSD of the best alignment
print(f'RMSD of best alignment: {aligner.rms} A')

RMSD of best alignment: 1.8158063761069152 A


## 6) Exercises (ungraded)

1. Pick a PDB of interest (protein or complex) and summarize:
   - number of chains
   - number of residues
   - any hetero residues (ligands/ions)
2. Make a visualization that clearly shows:
   - secondary structure (cartoon)
   - and ligand/hetero atoms (sticks), if present
3. **Sequence-alignment-guided RMSD**
   - Run Section 5
   - Report:
     - alignment score
     - number of matched residues
     - RMSD over matched Cα atoms
   - Make a screenshot of the superposition visualization

**If this were an assignment submission:** Save your work to Drive, and upload your `.ipynb` as the submission.
